# Well balanced second order MUSCL Scheme for the Shallow Water Equations

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import muscl2

## MUSCL reconstruction 
Using $U=(h,hu)$ the first reconstruction consists of
* Obtain reconstructed values of conserved variables $U_{i,r},U_{i,l}$ 
* Reconstruct free surface elevation $H_{i,l}$ and $H_{i,r}$ where $H=h+d$
* Obtain topography values $d_{i,l}$ and $d_{i,r}$, where $d=H-h$, con $H$ y $h$ from the reconstruction


In [6]:
def minmod(slope1, slope2):
    if (slope1>0 and slope2 > 0):
        return min(slope1,slope2)
    if (slope1<0 and slope2<0):
        return max(slope1,slope2)
    return 0.
minmod = np.vectorize(minmod)

In [10]:
def musclrecontr(U0,U1,U2,d0,d1,d2,dx=1):
    """
        Receives states U=(h,hu) and topography d, from a cell (say 1) 
        and its first neighbors (0 and 2) 
        and returns muscl reconstruction U_{il} and U_{ir} and
        d_{il}, d_{ir} of conserved variables and bathymetry at
        cell boundaries
        
        U0,U1,U2,d0,d1,d2 are 
        
        dx is irrelevant for regular grids
        but is kept for code semantic and mantainability
    """
    ###TODO ** Se podría mejorar: 
    ###hacer 1 función que calcule reconstrucción muscl de un arreglo cualquiera,
    ### y llamar a esa dentro de esta **

    #conserved variables u
    slope1 = (U1-U0)/dx
    slope2 = (U2-U1)/dx
#     slope = map(minmod,slope1,slope2)
    slope = minmod(slope1,slope2)
    Uil = U1-0.5*dx*slope
    Uir = U1+0.5*dx*slope
    
    #free surface H
    H0 = U0[0]+d0
    H1 = U1[0]+d1
    H2 = U2[0]+d2
    
    Hslope1 = (H1-H0)/dx
    Hslope2 = (H2-H1)/dx
    Hslope = minmod(Hslope1,Hslope2)
    Hil = H1-0.5*dx*Hslope
    Hir = H1+0.5*dx*Hslope
    
    #topography d
    d_il = Hil - uil[0]
    d_ir = Hir - uir[0]
    
    return d_il, d_ir , Uil, Uir

In [11]:
def getMusclReconstr(d,h,hu):
    """
    Receives d (1,nx+4) and h,hu (1,nx+4)
    and returns dl,dr,ul,ur (1\times nx+2)
    for cells -1 to nx (innercells + 1ghost at each end)
    """
    nx = d.shape[0]-4
    dl = np.zeros((nx+2,))
    dr = np.zeros((nx+2,))
    hl = np.zeros((nx+2,))
    hr = np.zeros((nx+2,))
    hul = np.zeros((nx+2,))
    hur = np.zeros((nx+2,))
    for i in range(1,nx+3,1):
        u0 = np.array([h[i-1],hu[i-1]])
        u1 = np.array([h[i],hu[i]])
        u2 = np.array([h[i+1],hu[i+1]])
        d0 = d[i-1]
        d1 = d[i]
        d2 = d[i+1]
        dl[i-1], dr[i-1], (hl[i-1],hul[i-1]), (hr[i-1],hur[i-1]) = musclrecontr(u0,u1,u2,d0,d1,d2)
    return dl, dr, hl, hr, hul, hur        

## Hydrostatic reconstruction
* Interface topography values are defined as $$d_{i+1/2} = max(d_{i,r}, d_{i+1,l})$$
* And water height at each side of interfaces as

$$h_{i+1/2-} = max(0, h_{i,r} +d_{i,r} - d_{i+1/2})$$
  $$h_{i+1/2+} = max(0, h_{i+1,l}+d_{i+1,l}-d_{i+1/2})$$
  
* Finally, conserved values at each side of the interface

$$ U_{i+1/2-} = (h_{i+1/2-}, h_{i+1/2-} u_{i,r})^t$$
$$ U_{i+1/2+} = (h_{i+1/2+}, h_{i+1/2+} u_{i+1,l})^t$$

In [5]:
muscl2.getMusclReconstr?